You're a consultant for DelFalco's Italian Restaurant. The owner asked you to identify whether there are any foods on their menu that diners find disappointing.

The business owner suggested you use diner reviews from the Yelp website to determine which dishes people liked and disliked. You pulled the data from Yelp. Before you get to analysis, run the code cell below for a quick look at the data you have to work with.

 You could group reviews by what menu items they mention, and then calculate the average rating for reviews that mentioned each item. You can tell which foods are mentioned in reviews with low scores, so the restaurant can fix the recipe or remove those foods from the menu.

In [3]:
import pandas as pd

In [4]:
data = pd.read_json("restaurant.json")

In [5]:
data.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
109,lDJIaF4eYRF4F7g6Zb9euw,lb0QUR5bc4O-Am4hNq9ZGg,r5PLDU-4mSbde5XekTXSCA,4,2,0,0,I used to work food service and my manager at ...,2013-01-27 17:54:54
1013,vvIzf3pr8lTqE_AOsxmgaA,MAmijW4ooUzujkufYYLMeQ,r5PLDU-4mSbde5XekTXSCA,4,0,0,0,We have been trying Eggplant sandwiches all ov...,2015-04-15 04:50:56
1204,UF-JqzMczZ8vvp_4tPK3bQ,slfi6gf_qEYTXy90Sw93sg,r5PLDU-4mSbde5XekTXSCA,5,1,0,0,Amazing Steak and Cheese... Better than any Ph...,2011-03-20 00:57:45
1251,geUJGrKhXynxDC2uvERsLw,N_-UepOzAsuDQwOUtfRFGw,r5PLDU-4mSbde5XekTXSCA,1,0,0,0,Although I have been going to DeFalco's for ye...,2018-07-17 01:48:23
1354,aPctXPeZW3kDq36TRm-CqA,139hD7gkZVzSvSzDPwhNNw,r5PLDU-4mSbde5XekTXSCA,2,0,0,0,"Highs: Ambience, value, pizza and deserts. Thi...",2018-01-21 10:52:58


In [6]:
menu = ["Cheese Steak", "Cheesesteak", "Steak and Cheese", "Italian Combo", "Tiramisu", "Cannoli",
        "Chicken Salad", "Chicken Spinach Salad", "Meatball", "Pizza", "Pizzas", "Spaghetti",
        "Bruchetta", "Eggplant", "Italian Beef", "Purista", "Pasta", "Calzones",  "Calzone",
        "Italian Sausage", "Chicken Cutlet", "Chicken Parm", "Chicken Parmesan", "Gnocchi",
        "Chicken Pesto", "Turkey Sandwich", "Turkey Breast", "Ziti", "Portobello", "Reuben",
        "Mozzarella Caprese",  "Corned Beef", "Garlic Bread", "Pastrami", "Roast Beef",
        "Tuna Salad", "Lasagna", "Artichoke Salad", "Fettuccini Alfredo", "Chicken Parmigiana",
        "Grilled Veggie", "Grilled Veggies", "Grilled Vegetable", "Mac and Cheese", "Macaroni",  
         "Prosciutto", "Salami"]

In [9]:
import spacy
from spacy.matcher import PhraseMatcher


In [8]:

index_of_review_to_test_on = 14
text_to_test_on = data.text.iloc[index_of_review_to_test_on]

nlp = spacy.blank('en')



review_doc = nlp(text_to_test_on)
matcher = PhraseMatcher(nlp.vocab, attr='LOWER')

menu_tokens_list = [nlp(item) for item in menu]

matcher.add("MENU",            # Just a name for the set of rules we're matching to
            None,              # Special actions to take on matched words
            *menu_tokens_list
           )

# Find matches in the review_doc
matches = matcher(review_doc)



In [11]:
 for match in matches:
    print(f"Token number {match[1]}: {review_doc[match[1]:match[2]]}")

Token number 2: Purista
Token number 16: prosciutto
Token number 58: meatball


In [13]:
from collections import defaultdict

# item_ratings is a dictionary of lists. If a key doesn't exist in item_ratings,
# the key is added with an empty list as the value.
item_ratings = defaultdict(list)

for idx, review in data.iterrows():
    doc = nlp(review.text)
    # Using the matcher from the previous exercise
    matches = matcher(doc)
    
    # Create a set of the items found in the review text
    found_items = set(doc[match[1]:match[2]] for match in matches)
    
    # Update item_ratings with rating for each item in found_items
    # Transform the item strings to lowercase to make it case insensitive
    for item in found_items:
        item_ratings[str(item).lower()].append(review.stars)

In [14]:
mean_ratings = {}

for item, ratings in item_ratings.items():
    mean_ratings[item] = sum(ratings) / len(ratings)

def find_key(value, dic):
    for key in dic.keys():
        if dic[key] == value:
            return key
    return None 
    
# Find the worst item, and write it as a string in worst_text. This can be multiple lines of code if you want.
worst_item = find_key(min(mean_ratings.values()), mean_ratings)


worst_text = str(worst_item)

In [16]:
mean_ratings = {item: sum(ratings)/len(ratings) for item, ratings in item_ratings.items()}
worst_item = sorted(mean_ratings, key=mean_ratings.get)[0]

In [18]:

print(worst_item)
print(mean_ratings[worst_item])

chicken cutlet
3.5454545454545454


In [20]:
counts = {item: len(ratings) for item, ratings in item_ratings.items()}

item_counts = sorted(counts, key=counts.get, reverse=True)
for item in item_counts:
    print(f"{item:>25}{counts[item]:>5}")

                    pizza  358
                    pasta  255
                 meatball  163
              cheesesteak  146
                  calzone  110
                 eggplant   95
                  cannoli   89
             cheese steak   88
                  lasagna   83
                  purista   67
               prosciutto   63
             chicken parm   58
          italian sausage   57
             garlic bread   46
                  gnocchi   45
                spaghetti   41
                 calzones   38
                   pizzas   33
                   salami   32
            chicken pesto   30
             italian beef   29
                 tiramisu   27
                     ziti   26
            italian combo   22
         chicken parmesan   21
       chicken parmigiana   18
           mac and cheese   18
               portobello   18
                 pastrami   16
           chicken cutlet   11
         steak and cheese    9
               roast beef    7
       f

In [21]:
sorted_ratings = sorted(mean_ratings, key=mean_ratings.get)

print("Worst rated menu items:")
for item in sorted_ratings[:10]:
    print(f"{item:20} Ave rating: {mean_ratings[item]:.2f} \tcount: {counts[item]}")
    
print("\n\nBest rated menu items:")
for item in sorted_ratings[-10:]:
    print(f"{item:20} Ave rating: {mean_ratings[item]:.2f} \tcount: {counts[item]}")

Worst rated menu items:
chicken cutlet       Ave rating: 3.55 	count: 11
turkey sandwich      Ave rating: 3.80 	count: 5
spaghetti            Ave rating: 3.85 	count: 41
italian combo        Ave rating: 3.91 	count: 22
eggplant             Ave rating: 3.97 	count: 95
italian beef         Ave rating: 4.00 	count: 29
tuna salad           Ave rating: 4.00 	count: 5
garlic bread         Ave rating: 4.02 	count: 46
meatball             Ave rating: 4.08 	count: 163
portobello           Ave rating: 4.11 	count: 18


Best rated menu items:
prosciutto           Ave rating: 4.62 	count: 63
purista              Ave rating: 4.64 	count: 67
chicken salad        Ave rating: 4.67 	count: 6
pastrami             Ave rating: 4.69 	count: 16
reuben               Ave rating: 4.80 	count: 5
steak and cheese     Ave rating: 4.89 	count: 9
artichoke salad      Ave rating: 5.00 	count: 5
fettuccini alfredo   Ave rating: 5.00 	count: 6
turkey breast        Ave rating: 5.00 	count: 1
corned beef          Ave ra